In [29]:
import numpy as np
import pandas as pd

def gini_impurity(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini


In [30]:

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_impurity(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [31]:

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def find_accuracy(y_test,y_pred):
    n = len(y_test)
    count = 0
    for i in range(n):
        if y_test[i] == y_pred[i]: 
            count += 1  
    return (count/n)*100

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [32]:

def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

dataset = pd.read_csv('Datasets/IRIS_2.csv').values
tree = build_tree(dataset, 3, 1)
prediction = []
y_test = dataset[100:110]
for row in y_test:
    prediction.append(predict(tree, row))
    print(f"Expected: {row[-1]}, Predicted: {prediction[-1]}")

accuracy_score = find_accuracy([row[-1] for row in y_test] , prediction)
print(f"Accuracy: {accuracy_score}%")

Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-versicolor
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Expected: Iris-virginica, Predicted: Iris-virginica
Accuracy: 90.0%


In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv('Datasets/IRIS_2.csv')

# Assuming the last column is the label
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(max_depth=3, random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Display the first few predictions
for i in range(5):
    print(f"Predicted: {y_pred[i]}, Actual: {y_test.iloc[i]}")


Accuracy: 1.0
Predicted: Iris-versicolor, Actual: Iris-versicolor
Predicted: Iris-setosa, Actual: Iris-setosa
Predicted: Iris-virginica, Actual: Iris-virginica
Predicted: Iris-versicolor, Actual: Iris-versicolor
Predicted: Iris-versicolor, Actual: Iris-versicolor
